In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys

#
from tqdm import tqdm, trange

#
import opexebo


Autosaving every 180 seconds


<ipython-input-1-432d4c01ef2b>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [6]:
###################################################################
############### COMPUTE PLACE FIELD OF SINGLE NEURON ##############
###################################################################
#
def load_locs_traces(fname,
                     arena_size):
    
    #
    locs = np.load(fname)
    print (locs.shape)

    ####################### COMPUTE SPATIAL OCCUPANCY ###########################
    times = np.arange(locs.shape[0])

    #
    min_x = np.min(locs[:,0])
    max_x = np.max(locs[:,0])

    min_y = np.min(locs[:,1])
    max_y = np.max(locs[:,1])

    #
    locs[:,0] = (locs[:,0]-min_x)/(max_x-min_x)*arena_size[0]
    locs[:,1] = (locs[:,1]-min_y)/(max_y-min_y)*arena_size[1]

    ####################### LOAD SPIKES ########################
    fname = '/media/cat/4TB1/donato/nathalie/DON-007050/FS1/binarized_traces.npz'
    bin_ = np.load(fname,
                   allow_pickle=True)

    #
    traces = bin_['F_upphase']
    print ("traces: ", traces.shape)

    #
    return locs, traces, times

#############################################################
#############################################################
#############################################################
fname = ('/media/cat/4TB1/donato/nathalie/DON-007050/FS1/DON-007050_20211022_'+
         'TR-BSL_FS1-ACQDLC_resnet50_open_arena_white_floorSep8shuffle1_200000_locs.npy')

#
arena_x = [300,1550]
arena_y = [175,1400]
arena_size = [80,80]
arena_shape = 'square'
bin_width = 2.5

#
locs, traces, times = load_locs_traces(fname,
                                       arena_size)

#
spatial_occupancy, coverage, bin_edges = opexebo.analysis.spatial_occupancy(times, 
                                                                            locs.T, 
                                                                            arena_size,
                                                                            arena_shape = arena_shape,
                                                                            bin_width = bin_width)
#################################################
#################################################
#################################################
print (spatial_occupancy.shape)

cell_id = 6
traces = traces[cell_id]

# find times when cell is spking
t = np.where(traces>0)[0]
print (t)

# make sure no indexes go over time;
idx = np.where(t<36000)[0]
t=t[idx]


#
x = locs[t,0]
y = locs[t,1]

spikes_tracking = np.vstack((t,x,y))

######################## MAKE RATE MAP ###########################
rate_map = opexebo.analysis.rate_map(spatial_occupancy,
                                     spikes_tracking,
                                     arena_size)

_, place_field = opexebo.analysis.place_field(rate_map)
    
# #################################################
plt.figure()
ax=plt.subplot(2,2,1)
plt.plot(locs[:,0],
         locs[:,1])
ax.set_aspect('equal', adjustable='box')
plt.title('locations')

#
ax=plt.subplot(2,2,2)
plt.imshow(spatial_occupancy)

#plt.ylim(0,mmap.shape[1])
#plt.xlim(1,mmap.shape[0])
plt.title('occupancy map')

#
ax=plt.subplot(2,2,3)
plt.imshow(rate_map)
plt.title('rate_map')


#
ax=plt.subplot(2,2,4)
plt.imshow(place_field)
plt.title('place field')
           
plt.show()

#

(36000, 2)
traces:  (476, 36025)
(32, 32)
[ 2386  2387  2388  2389  2390  2391  2392  2393  2394  2395  2396  2397
  2398  2399  2400  2401  2402  2403  2404  2405  2406  2407  2408  2409
  2410  2411  2412  2413  2414  2415  2416  2417  2418  2419  2420  2421
  2422  2423  2424  2425  2426  2427  2428  2429  2430  2431  2432  2433
  2434  2435  2436  2437  2438  6453  6454  6455  6456  6457  6458  6459
  6460  6461  6462  6463  6464  6465  6466  6467  6468  6469  6470  6471
  6472  6473  6474  6475  6476  6477  6478  9813  9814  9815  9816  9817
  9818  9819  9820  9821  9822  9823  9824  9825  9826  9827  9828  9829
  9830  9831  9832  9833  9834  9835  9836  9837  9838  9839  9840  9841
  9842  9843  9844  9845  9846  9847  9848  9849  9850  9851  9852  9853
  9854  9855  9856  9857  9858  9859  9860  9861  9862  9863  9864  9865
  9866  9867  9868  9869  9870  9871  9872  9873  9874  9875  9876  9877
  9878  9879  9880  9881  9882  9883  9884  9885  9886  9887  9888  9889
  9890  9

In [4]:
###############################################################
###############################################################
###############################################################

import opexebo as op

#
def load_locs_traces(fname,
                     arena_size):
    
    #
    locs = np.load(fname)
    print (locs.shape)

    ####################### COMPUTE SPATIAL OCCUPANCY ###########################
    times = np.arange(locs.shape[0])

    #
    min_x = np.min(locs[:,0])
    max_x = np.max(locs[:,0])

    min_y = np.min(locs[:,1])
    max_y = np.max(locs[:,1])

    #
    locs[:,0] = (locs[:,0]-min_x)/(max_x-min_x)*arena_size[0]
    locs[:,1] = (locs[:,1]-min_y)/(max_y-min_y)*arena_size[1]

    ####################### LOAD SPIKES ########################
    fname = '/media/cat/4TB1/donato/nathalie/DON-007050/FS1/binarized_traces.npz'
    bin_ = np.load(fname,
                   allow_pickle=True)

    #
    traces = bin_['F_upphase']
    print ("traces: ", traces.shape)

    #
    return locs, traces, times

#############################################################
#############################################################
#############################################################
fname = ('/media/cat/4TB1/donato/nathalie/DON-007050/FS1/DON-007050_20211022_'+
         'TR-BSL_FS1-ACQDLC_resnet50_open_arena_white_floorSep8shuffle1_200000_locs.npy')

#
arena_x = [300,1550]
arena_y = [175,1400]
arena_size = [80,80]
arena_shape = 'square'
bin_width = 2.5

#
locs, traces, times = load_locs_traces(fname,
                                       arena_size)


occ_map, coverage, bin_edges = op.analysis.spatial_occupancy(times, 
                                                             locs.T, 
                                                             arena_size, 
                                                             bin_width=bin_width)

#
ids = [28,30,32,46,
       122,132,174,208,
       209,227,236,238,
       289,312,318,390]


cell_id = ids[5]
traces = traces[cell_id]

# find times when cell is spking
t = np.where(traces>0)[0]

# make sure no indexes go over time;
idx = np.where(t<36000)[0]
t=t[idx]


#
x = locs[t,0]
y = locs[t,1]

spikes_tracking = np.vstack((t,x,y))

#
rm = op.analysis.rate_map(occ_map, 
                          spikes_tracking, 
                          bin_edges=bin_edges, 
                          arena_size=arena_size, 
                         # limits=limits
                         )

sigma = 1.0
rms = op.general.smooth(rm, 
                        sigma)

fields, fields_map = op.analysis.place_field(rms, 
                                            search_method='sep'
                                            )

#
print ("fiedsL ", fields)
print ("fields map: ", fields_map)

##########################################################################
##########################################################################
##########################################################################

#
plt.figure()
ax=plt.subplot(131)
plt.imshow(occ_map)
plt.title("occ_map")
#
ax=plt.subplot(132)
plt.imshow(rm)
plt.title("rm")

#
ax=plt.subplot(133)
plt.imshow(fields_map)
plt.title("place field")
plt.suptitle("cell id: "+str(cell_id))
plt.show()


(36000, 2)
traces:  (476, 36025)
fiedsL  []
fields map:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
####################################################
####################################################
####################################################.

def get_place_cells(traces,
                    spatial_occupancy,
                    plotting=False):

    #
    fields = []
    rate_maps = []
    empty_field = np.zeros((32,32))+np.nan
    for cell_id in trange(traces.shape[0]):
        
        #
        traces1 = traces[cell_id]

        # find times when cell is spking
        t = np.where(traces1>0)[0]

        #
        idx = np.where(t<36000)[0]
        t=t[idx]
        
        
        #
        x = locs[t,0]
        y = locs[t,1]
        #print (t.shape, xy.shape)

        spikes_tracking = np.vstack((t,x,y))
        #print (spikes_tracking.shape)


        #
        try:
            rate_map = opexebo.analysis.rate_map(spatial_occupancy,
                                                 spikes_tracking,
                                                 arena_size)
        except:
            print (cell_id, " errror...")
            rate_maps.append(empty_field)
            fields.append(empty_field)

        #
        #print (rate_map.shape)
        
        #
        rate_maps.append(rate_map)

        #
        if plotting:
            plt.figure()
            ax=plt.subplot(1,1,1)
            plt.imshow(rate_map)

            plt.show()


        #
        _, place_field = opexebo.analysis.place_field(rate_map)

        #
        fields.append(place_field)
        
        #
        if plotting:
            plt.figure()
            plt.imshow(place_field)

            plt.show()

    return np.array(fields), rate_maps

fname = '/media/cat/4TB1/donato/nathalie/DON-007050/FS1/binarized_traces.npz'
bin_ = np.load(fname,
               allow_pickle=True)

traces = bin_['F_upphase']
print ("traces: ", traces.shape)

fields, rate_maps = get_place_cells(traces,
                            spatial_occupancy,
                            plotting=False)
print ("fields: ", fields.shape)



traces:  (476, 36025)


 29%|██▉       | 138/476 [00:00<00:00, 347.36it/s]

101  errror...


 60%|██████    | 286/476 [00:00<00:00, 357.46it/s]

241  errror...


 84%|████████▍ | 399/476 [00:01<00:00, 339.92it/s]

361  errror...


100%|██████████| 476/476 [00:01<00:00, 337.32it/s]

fields:  (479, 32, 32)


In [54]:
plt.figure()
for ctr,k in enumerate(range(0,100,1)):
    ax=plt.subplot(10,10,ctr+1)
    plt.imshow(rate_maps[k])
    plt.xticks([])
    plt.yticks([])
    plt.title(k, pad=0.9, fontsize=10)
plt.suptitle("Rate maps - UPPHASE")
plt.show()

In [59]:
plt.figure()
for ctr,k in enumerate(range(0,100,1)):
    ax=plt.subplot(10,10,ctr+1)
    plt.imshow(fields[k])
    plt.xticks([])
    plt.yticks([])
    plt.title(k, pad=0.9, fontsize=10)
plt.suptitle("Place fields - UPPHASE")
plt.show()

In [32]:
ids = [28,30,32,46,
       122,132,174,208,
       209,227,236,238,
       289,312,318,390]
plt.figure()
for ctr,k in enumerate(ids):
    ax=plt.subplot(4,4,ctr+1)
    plt.imshow(fields[k])
    plt.xticks([])
    plt.yticks([])
    plt.title(k)
plt.show()

